In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:

# IPR Data
ipr = pd.DataFrame({
    'Pwf': [4000,3500,3000,2500,2000,1500,1000,500,14.7],
    'Q':   [0,1999,3094,3902,4512,4963,5275,5458,5519]
})

# TPR Data
q = np.arange(1000,6500,500)
p_190 = np.array([1334,1400,1487,1592,1712,1843,1984,2132,2287,2446,2609])
p_2375 = np.array([1298,1320,1351,1390,1435,1487,1545,1609,1677,1749,1824])
p_2875 = np.array([1286,1294,1305,1319,1336,1356,1378,1403,1431,1461,1493])

tpr_base = {
    '1.90"': 'Pwf(1.90" tbg)',
    '2.375"': 'Pwf(2.375"tbg)',
    '2.875"': 'Pwf(2.875"tbg)'
}




In [ ]:
# Interpolation function
def interpolate_pressure(q_vals, pwf_vals, q_range):
    return np.interp(q_range, q_vals, pwf_vals)

def plot_nodal(wc, gor, skin, tubing_id):
    plt.figure(figsize=(9, 6))

    # Plot IPR
    plt.plot(ipr['Q'], ipr['Pwf'], label="IPR", color='black', linewidth=2)

    # Adjust TPR
    base_tpr_col = tpr_base[tubing_id]

    base_tpr_values = tpr[base_tpr_col]

    adj_factor = 1 + 0.002 * wc + 0.001 * gor + 0.05 * skin
    tpr_adj = base_tpr_values * adj_factor
    tpr_adj = np.clip(tpr_adj, a_min=0, a_max=None)


    # Plot TPR
    plt.plot(q, tpr_adj, label=f"TPR {tubing_id}", color='blue', linewidth=2)

    # Interpolate
    q_interp = np.linspace(q.min(), q.max(), 1000) # Interpolate over the range of q
    ipr_interp = interpolate_pressure(ipr['Q'], ipr['Pwf'], q_interp)
    tpr_interp = interpolate_pressure(q, tpr_adj, q_interp)

    # Difference and thresholdn
    min_len = min(len(ipr_interp), len(tpr_interp))
    diff = np.abs(ipr_interp[:min_len] - tpr_interp[:min_len])
    min_diff = np.min(diff)
    idx = np.argmin(diff)

    # If curves are close enough, mark intersection
    if min_diff < 25:
        q_opt = q_interp[idx]
        p_opt = ipr_interp[idx]
        plt.plot(q_opt, p_opt, 'ro', label=f'Optimum Rate ≈ {q_opt:.1f} Mscf/D\n@ {p_opt:.1f} psia')
    else:
        print(" No valid intersection found between IPR and TPR curves.")


    plt.xlabel('Flow Rate (Mscf/D)')
    plt.ylabel('Flowing Bottomhole Pressure (psia)')
    plt.title('Nodal Analysis with Sensitivity & Optimum Rate')
    plt.grid(True)
    plt.legend()
    # plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()


# Sliders
wc_slider = widgets.IntSlider(value=30, min=0, max=90, step=5, description='Water Cut (%)')
gor_slider = widgets.IntSlider(value=800, min=100, max=2000, step=100, description='GOR (scf/bbl)')
skin_slider = widgets.FloatSlider(value=0, min=-2.0, max=10.0, step=0.5, description='Skin')
tubing_slider = widgets.Dropdown(options=["1.90\"", "2.375\"", "2.875\""], value="2.375\"", description="Tubing ID")

interact(plot_nodal, wc=wc_slider, gor=gor_slider, skin=skin_slider, tubing_id=tubing_slider)